### Django Form
- 사용장의 비정상적인, 악의적인 요청이 있다는 것을 생각해야 함
- 따라서 유효성 검증이 필요한데 Django From이 이것을 간단하게 진행하도록 해줌
- Model 클래스를 선언하는것과 비슷하다.
  - 비슷한 이름의 필드 타입을 많이 가지고 있다.(이름이 같을 뿐 같은 필드 아님)
  - Model과 마찬가지로 상속을 통해 선언(forms 라이브러리의 Form 클래스 상속받음)
- form에는 TextField가 존재하지 않음.
- 파일 이름은 forms.py가 아니어도 되고 models.py나 다른 어디에도 작성 가능하나 유지보수, 관리의 관점에서 관행적으로 forms.py에 작성하는것을 권장

``` python
from django import forms

class ArtifcleForm(forms.Form):
    title = forms.CharField(max_length=10)
    content = forms.CharField()
```

- 이것을 사용하려면 views.py에서 form을 넘겨주도록 수정
- 이후 html파일에서도 해당 부분을 수정 {{ form }} 하나로도 충분
- CharField 하나가 input과 label로 자동 렌더링
- 속성값(.as_p, .as_ul, .as_table)으로 줄바꿈 등 구현, 각각의 태그로 감싸져서 렌더링된다.
- input을 라디오, 체크박스 등으로 바꾸거나 textarea로 바꾸고 싶을때 widget
  - forms.CharField(widget=forms.Textarea)
  - input요소 렌더링을 담당 (단순한 출력 부분을 담당)
  - widgets은 반드시 form fiels에 할당 됨
  - 유효성 검증과 아무런 관련 없음

### ModelForm
- models.py에 쓴 것을 form에 맵핑하기 위해 또 재정의 해야함
- Model을 이요해 Form Class를 만들 수 있는 helper class
- form과 똑같은 방식으로 view 함수에서 이용

- Meta 클래스 안에 model, fields 변수를 이용해 작성
``` python
class ArticleForm(forms.ModelForm):

    class Meta:
        model = Article # 어떤 모델을 기반으로 할지
        fields = '__all__'  # 모델 필드 중 어떤 것을 출력할지
        # 원래 튜플로 작성 함. ('title', 'content')
```
- exlude를 통해 특정 필드를 제외시킬 수 있음. 작성은 튜플로
- Meta data란 데이터에 대한 데이터

### ModelForm으로 인한 view함수 구조 변화 -CREATE
``` python
def create(request):

    # ModelForm 이용시 한 줄로 다 받음
    form = ArticleForm(request.POST)
    if form.is_valid(): # 유효성 검사
        article = form.save()
        return redirect('articles:detail', article.pk)
    return redirect('articles:new')
```

### save()
- form 인스턴스에 바인딩(들어온)된 데이터를 통해 데이터베이스 객체를 만들고 저장
- instance 여부를 통해 생성할지, 수정할지 결정
  - 예시: form = ArticleForm(request.POST, instance=article)

### is_valid()
- form.errors에 왜 에러가 떴는지(유효성 검사에 실패했는지) 딕셔너리 형태로 담아줌
- print(f'에러: {form.errors}')
- 원래 아무 데이터 안담겨있지만 is_valid()를 통해서만 쓸 수 있음

### ModelForm으로 인한 view함수 구조 변화 - UPDATE
``` python
def edit(request, pk):
    article = Article.objects.get(pk=pk)
    form = ArticleForm(instance=article)
    context = {
        'article': article,
        'form':form,
    }
    return render(request, 'articles/edit.html', context)


def update(request, pk):
    article = Article.objects.get(pk=pk)
    form = ArticleForm(request.POST, instance=article)
    if form.is_valid():
        form.save()
        return redirect('articles:detail', article.pk)
    context = {
        'form':form,
    }
    return render(request, 'articles/edit.html', context)
```


### Form과 ModelForm
- Form은 사용자로부터 받는 데이터가 DB와 연관되어 있지 않는 경우
- DB에 영향을 미치지 않고 단순 데이터만 사용되는 경우(로그인 - 별도 DB에 저장 안함)
- ModelForm은 받는 데이터가 DB와 연관되어있는 경우
- 데이터의 유효성 검사가 끝나면 데이터를 각각 어떤 레코드에 맵핑애햐할지 이미 알고있기 때문에 곧바로 save()호출 가능


### widget 활용하기
- 작성할 때에 Meta의 밖으로 빼서 별도로 작성
- attrs(attributes) 안에 여러 속성과 값을 딕셔너리 형태로 넣기
``` python
class ArticleForm(forms.ModelForm):
    title = forms.CharField(
        label='제목',
        widget=forms.TextInput(
            # attributes
            attrs={
                'class': 'my-title',
                'placeholder': 'Enter the title',
                'maxlength': 10,
            }
        ),
    )
```

### Handling HTTP request
- new-create(생성을 위함), edit-update(수정을 위함)
- new와 edit은 GET 요청에 대한 처리만을(페이지렌더링)
- create와 update는 POST요청에 대한 처리만을(DB조작)
- 이 공통점과 차이점을 기반으로 하나의 view함수에서 method에 따라 로직이 분리되도록 변경
- new와 create를 합침
``` python
def create(request):
    # new 흡수
    if request.method == 'POST':
        # 기존의 create 코드
        form = ArticleForm(request.POST)
        if form.is_valid(): # 유효성 검사
            article = form.save()
            return redirect('articles:detail', article.pk)
    else:
        # 기존의 new 코드
        form = ArticleForm()
    context = {
        'form' : form,
    }
    return render(request, 'articles/new.html', context)
```
- 기존에 쓰이던 new 전부 create로 바꿔주기



- edit과 update를 합침
``` python
def update(request, pk):
    # edit 흡수
    article = Article.objects.get(pk=pk)
    if request.method == 'POST':
        form = ArticleForm(request.POST, instance=article)
        if form.is_valid():
            form.save()
            return redirect('articles:detail', article.pk)
    else:
        form = ArticleForm(instance=article)
    context = {
        'article': article,
        'form':form,
    }
    return render(request, 'articles/update.html', context)
```
- 기존에 쓰이던 eidt 전부 update로 바꿔주기


- DELETE도 결국 DB를 조작하는 형태
- 하지만 GET이든 POST든 함수만 호출되면 지워짐
- 이것을 POST일때만 지우도록 수정 
``` python
def delete(request, pk):
    if request.method =='POST':
        article = Article.objects.get(pk=pk)
        article.delete()
    return redirect('articles:index')
```


- POST, GET 말고 다른 메서드도 존재하므로 if문에 POST를 적어주는게 맞음

### View decorators
- 기존에 작성된 함수에 기능을 추가할때, 해당 함수를 수정하지 않고 기능을 추가해주는 함수
- Allowed HTTP methods Django에서 지원하는 메소드 대표 3가지
- require_http_methods(), require_POST(), require_safe()
``` python
from django.views.decoreators.http import require_safe

@require_safe   # get인 요청에 대해서만 이 밑의 함수 실행, 아닐경우 405
def index(request):
```
- 같은 방식으로 require_POST()도 쓸 수 있음
``` python
@require_http_methods(['GET', 'POST'])
```


### Rendering fields manually
- 공식문서 django form
- html에서 {{form.as_p}}하나가 아니라 따로 쪼개서 관리할 수 있음.
- 양이 많다면 반복으로 처리 가능(Looping over the form's fields)
- bootstrap 적용하기
  - forms.py에 class 넣기
  - 다른 방법으로는 djangto bootstrap 5 검색해서 공식문서대로!